In [ ]:
import cv2
import time
import requests
import os
import time
from os import system
from dotenv import load_dotenv

In [ ]:
load_dotenv(os.path.join(os.path.dirname(os.path.curdir), '.env'))

In [ ]:
# Set up the Hugging Face token here
API_URL = "https://api-inference.huggingface.co/models/Salesforce/blip-image-captioning-large"
API_TOKEN = os.environ.get("HUGGINGFACE_TOKEN")
headers = {"Authorization": "Bearer {}".format(API_TOKEN)}

In [ ]:
def speak(text):
    system(f'say "{text}"')

In [ ]:
def query(frame):
    with open(frame, "rb") as f:
        data = f.read()
    response = requests.post(API_URL, headers=headers, data=data)
    return response.json()

In [ ]:
# Init camera
cap = cv2.VideoCapture(0)

In [ ]:
last_capture_time = time.time()
filename = ""

In [ ]:
while True:
    ret, frame = cap.read()
    if ret == False:
        break

    current_time = time.time()
    time_diff = current_time - last_capture_time
    
    if time_diff >= 5:
        timestamp = time.strftime("%Y%m%d-%H%M%S")
        filename = f"frame_{timestamp}.jpg"
        cv2.imwrite(filename, frame)
        last_capture_time = current_time
        saved_frame = filename
        response = query(saved_frame)
        caption = response[0]["generated_text"]
        speak(caption)
        print(caption)
        os.remove(saved_frame)

    cv2.imshow("Video Frame", frame)
    key_pressed = cv2.waitKey(1) & 0xFF
    if key_pressed == ord('q'):
        break

In [ ]:
cap.release()
cv2.destroyAllWindows()